---
title: "Differential Equations II"
subtitle: ""
format: 
  html:
    toc: true
    code-fold: false
    page-layout: full
    fig-cap-location: bottom
    number-sections: true
    number-depth: 2
    html-to-math: katex
    html-math-method: katex
    callout-appearance: minimal
jupyter: python3
---

# Boundary-Value Problems
Our prototypical _boundary-value problem (BVP)_ will be a second-order ODE:
$$
w''(x) = f(x,w,w'),\quad w(a) = c, \quad w(b) = d.
$$
Since $w'(a)$ is not given, this is not an initial value problem. Here, the function value at the other end $w(b)$ is given. This is known as the _Dirichlet boundary condition_.
On the other hand, if $w'(b)$ is given instead, then we have the _Neumann boundary condition_.

Below, we will discuss two general approaches to solving this BVP: (a) treat it as a more complicated version of an initial-value problem, or (b) use the fact that we
know two boundary conditions and set up a method accordingly. We discuss these two approaches in turn.
